In [1]:
import pandas as pd
from pydub import AudioSegment
import math
from tqdm.notebook import tqdm

In [2]:
fil_list = pd.read_csv('fil_list.csv',index_col=0)
tracks_dir = './track/'
ten_minutes = 10*60*1000


In [3]:
fil_track = fil_list.copy()
fil_track.insert(1,'track','')
fil_track

,abbr,track,language
Кавказский пленник (1996),KP,,ru


In [4]:
for film in tqdm(fil_list.index):

    abbr = fil_list.loc[film]['abbr']
    language = fil_list.loc[film]['language']
    track_dir = tracks_dir + abbr
    track_full_path = track_dir + '/' + abbr + '.mp3'
    track = AudioSegment.from_mp3(track_full_path)
    beginning = 0
    track_len = len(track)
    counter = 1

    while beginning < track_len:

        next_end = beginning + ten_minutes
        end = next_end if next_end < track_len else track_len
        trim = track[beginning:end]
        trim_full_path = f'{track_dir}/{abbr}_{str(counter)}.mp3'
        file_handle = trim.export(trim_full_path,
                            format='mp3',
                            bitrate="256k",
                            tags={"album": abbr, "artist": f'No{counter}'})
        new_row = pd.DataFrame({'abbr':abbr,'track_num':trim_full_path,'language':language}, index=[film])
        abbr = fil_list.loc[film]['abbr']
        fil_track = pd.concat([fil_track.loc[:,:],new_row],)
        beginning += ten_minutes
        counter += 1

    print(f'Track {abbr} trimmed to {counter-1} parts')

  0%|          | 0/1 [00:00<?, ?it/s]

Track KP trimmed to 10 parts


In [5]:
fil_track.to_csv('fil_track.csv')